# Execution tutorial

This tutorial covers the basics of executing a quantum circuit using Classiq directly through the Python SDK. It is also possible to use the [Classiq Platform](https://platform.classiq.io) to execute quantum algorithms.

For this, we will start by synthesizing the following example, from the [synthesis tutorial](https://docs.classiq.io/latest/explore/tutorials/basic_tutorials/the_classiq_tutorial/synthesis_tutorial/):

In [1]:
from classiq import *


@qfunc
def main(x: Output[QNum[3]], y: Output[QNum]) -> None:
    allocate(x)
    hadamard_transform(x)
    y |= x**2 + 1


qprog = synthesize(main)
show(qprog)

Quantum program link: https://platform.classiq.io/circuit/2zManKvRhEsNL418zIGuXlatYV0


This quantum program evaluates the function $y(x) = x^2 + 1$, for all integers $x \in [0,7]$. To execute a quantum program and save its results in the Python SDK, create an `ExecutionSession`. To sample the states using this object, one can use `sample`:

In [2]:
with ExecutionSession(qprog) as es:
    results = es.sample()

The information from the outputs of the quantum circuit can be obtained in the form of a dataframe using the `dataframe` attribute:

In [3]:
results.dataframe

,x,y,count,probability,bitstring
0,3,10,287,0.140137,001010011
1,2,5,268,0.130859,000101010
2,5,26,266,0.129883,011010101
3,1,2,262,0.127930,000010001
4,4,17,253,0.123535,010001100
5,0,1,245,0.119629,000001000
6,6,37,236,0.115234,100101110
7,7,50,231,0.112793,110010111


The information displayed in the dataframe are:

* `counts` will output the number of counts of each state measured.
* `bitstring` are the bitstring that represent each state measured.
* `x` and `y` are the numerical representation of the states associated to it.
* `probability` are the probability associated with each each state measured.

## Backend selection

The backend of an execution is the hardware choice where the quantum program is executed. It can be a simulator or real hardware. To select a specific backend, it is necessary to import its correct Backend Preferences from `classiq.execution`. Check the different [Cloud Providers](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) and their backend preferences for execution.

In this section we will explore two different examples for fixation: 

### First example: Execution using the state vector simulator from Classiq

Since Classiq provides its own state vector simulator backend, we will use `ClassiqBackendPreferences` to define it as the state vector simulator. This information is provided on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/).

To define the quantum program's execution preferences, use `ExecutionPreferences` and then `set_quantum_program_execution_preferences`. In this example, we will perform a simulation with `num_shots=1` since the state vector simulator performs an exact simulation of the quantum program.

If no backend is defined on the preferences, then the [Classiq simulator](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/classiq-backends/) is selected.


In [4]:
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences

backend_preferences = ClassiqBackendPreferences(
    backend_name="simulator_statevector"
)  # Always check the Cloud Providers to correctly define the backend.

execution_preferences = ExecutionPreferences(
    num_shots=1, backend_preferences=backend_preferences
)

Now, execute the quantum program using `execute`.

In [5]:
with ExecutionSession(qprog, execution_preferences=execution_preferences) as es:
    results_statevector = es.sample()

The outputs of the quantum program can be obtained through the attributes `state_vector` and `parsed_state_vector` from `results.result_value()`. Below, we can see the amplitudes for the state `'x':0, 'y':1`, which can be represented by the bitstring '0000000000001000':

In [6]:
print(
    "Amplitude of the state 000001000: ",
    results_statevector.state_vector.get("000001000", None),
)

state = next(
    (s for s in results_statevector.parsed_state_vector if s.bitstring == "000001000"),
    None,
)
print("Amplitude of the state {'x':0, 'y':1}: ", state.amplitude if state else None)

Amplitude of the state 000001000:  None
Amplitude of the state {'x':0, 'y':1}:  None


The outputs from the execution obtained via statevector simulator will differ from the default simulator:

* `state_vector` will output a `dict` containing the bitstrings followed by its numerically evaluated amplitudes.
* `parsed_state_vector` will output a `list` of `SimulatedState`, each containing the values of `x` and `y` followed by its bitstrings and its numerically evaluated amplitudes.

### Second example: Execution on free access IBM Hardware

We will now execute our quantum program on `ibm_brisbane`, a quantum computer from IBM Quantum. For this, we need to check on the [Cloud Providers page](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/) for the [IBM Quantum Backends](https://docs.classiq.io/latest/user-guide/execution/cloud-providers/ibm-backends/). First, we need to do [hardware-aware synthesis](), i.e., synthesize the quantum program accordingly to the hardware that it will be executed. This can be done by specifying the backend using the `Preferences` function.

In [7]:
preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_brisbane"
)

qprog_IBM = synthesize(main, preferences=preferences)

On the Cloud Providers page, we can see that an access token from an IBM Quantum account is required. Using this information, it is possible to set our execution preferences:

In [ ]:
from classiq.execution import (
    ExecutionPreferences,
    IBMBackendPreferences,
    IBMBackendProvider,
)

ibm_provider = IBMBackendProvider(hub="ibm-q", group="open", Project="main")
ibm_backend_preferences = IBMBackendPreferences(
    backend_name="ibm_brisbane",
    access_token="your_job_id",
    provider=ibm_provider,
)

execution_preferences = ExecutionPreferences(
    num_shots=1000, backend_preferences=ibm_backend_preferences
)

In [ ]:
# Uncomment the following line to set the execution preferences and execute the quantum program on the modified backend.

# with ExecutionSession(qprog_IBM, execution_preferences) as es:
#    results = es.sample()

Its outputs will formatted in the same way of the outputs of the Classiq simulator:

In [15]:
results.dataframe

,x,y,count,probability,bitstring
0,6,2,7,0.007,000010110
1,6,21,7,0.007,010101110
2,7,44,7,0.007,101100111
3,6,16,6,0.006,010000110
4,7,32,6,0.006,100000111
...,...,...,...,...,...
430,7,60,1,0.001,111100111
431,1,63,1,0.001,111111001
432,2,63,1,0.001,111111010
433,5,63,1,0.001,111111101


When executing on hardware, it is often necessary to wait while your job is pending, i.e., wait for the other jobs scheduled to be executed on that machine to finish before running yours. However, if you don’t want your Python script to be blocked during this time, or if you want to store the job ID to retrieve the job later, allowing even the access of the results from different machines, by using `submit_sample` instead of `sample`.

In [21]:
with ExecutionSession(qprog) as es:
    results = es.submit_sample()
    job_ID = results.id

Once you have a job ID, it is possible to retrieve its execution data using `ExecutionJob`:

In [20]:
# Retrieving its information:
job_execution = ExecutionJob.from_id(job_ID)

job_execution.result_value().dataframe

,x,y,count,probability,bitstring
0,6,2,7,0.007,000010110
1,6,21,7,0.007,010101110
2,7,44,7,0.007,101100111
3,6,16,6,0.006,010000110
4,7,32,6,0.006,100000111
...,...,...,...,...,...
430,7,60,1,0.001,111100111
431,1,63,1,0.001,111111001
432,2,63,1,0.001,111111010
433,5,63,1,0.001,111111101
